## ORM

In [1]:
from sqlalchemy import create_engine
import MySQLdb

engine = create_engine("mysql://root:@localhost:3306/fish", echo=True)

In [73]:
# connection engine
conn = engine.connect()
# conn.execute() and engine.execute() are same

In [2]:
# sessions
from sqlalchemy.orm import sessionmaker
# session makes middle-ground between objects in Python and the engine
# that actually communicates with the database

Session = sessionmaker(bind=engine)
session = Session()

In [79]:
# creating tables
# Each class will be a table in our db and each attribute
# will be a column in this table. To map which table in 
# the db will be related to each class in our files, we 
# will use a SQLAlchemy system called Declarative


from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    password = Column(String(255))
    
    def __repr__(self):
        return f'User {self.name}'
    

# creating table in database
Base.meta.create_all(engine)

In [81]:
# adding new records

user = User(name="Pujan Dahal", password="12345678")
session.add(user)

print(user.id) # None because the new record is only flushed i.e. recorded
# for commit but not actually committed

session.commit() # it commits all previous pending operations that were flushed
print(user.id) # 9

None
9


In [94]:
# querying table

query = session.query(User).filter_by(name="Pujan Dahal")
query.count() # 2

query = session.query(User).filter(User.name=="John Snow").all()


In [95]:
# creating new table after initial create_all

class Product(Base):
    __tablename__ = 'products'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    

Product.__table__.create(engine) # creates new table in database

In [5]:
# creating foreign key relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import relationship

Base = declarative_base()

class Product(Base):
    __tablename__ = 'product'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    user_id = Column(Integer, ForeignKey('user.id'))
    user = relationship('User')
    
    def __repr__(self):
        return f"Product {self.name}"
        
    
class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    password = Column(String(255))
    products = relationship(Product, backref='users')
    
    
    def __repr__(self):
        return f"User {user.name}"
        

# create tables
Base.metadata.create_all(engine)

user = User(name='John')
product = Product(name='fogg', user=user)

session.add_all([user, product])
session.commit()
    



2022-09-15 13:43:58,196 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-09-15 13:43:58,198 INFO sqlalchemy.engine.base.Engine ()
2022-09-15 13:43:58,200 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-09-15 13:43:58,200 INFO sqlalchemy.engine.base.Engine ()
2022-09-15 13:43:58,202 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2022-09-15 13:43:58,204 INFO sqlalchemy.engine.base.Engine ()
2022-09-15 13:43:58,205 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2022-09-15 13:43:58,206 INFO sqlalchemy.engine.base.Engine ()
2022-09-15 13:43:58,208 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2022-09-15 13:43:58,209 INFO sqlalchemy.engine.base.Engine ()
2022-09-15 13:43:58,211 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2022-09-15 13:43:58,213 INFO sqlalchemy.engine.base.E